In [1]:
# read CCLE

import pandas as pd
import os
import gzip


# CCLE has the gene expression

# File path
folder = "./datasets/"
ccle_path = os.path.join(folder, "CCLE_RNAseq_rsem_genes_tpm_20180929.txt.gz")


if not os.path.exists(ccle_path):
    print("File not found. Please check the file path.")
else:
    print("File exists.")

# Read the gzipped file
ccle_data = pd.read_csv(ccle_path, compression='gzip', sep='\t', comment='#')

# Print the head of the dataset
print(ccle_data.head())


File exists.
              gene_id                                     transcript_ids  \
0  ENSG00000000003.10  ENST00000373020.4,ENST00000494424.1,ENST000004...   
1   ENSG00000000005.5                ENST00000373031.4,ENST00000485971.1   
2   ENSG00000000419.8  ENST00000371582.4,ENST00000371583.5,ENST000003...   
3   ENSG00000000457.9  ENST00000367770.1,ENST00000367771.6,ENST000003...   
4  ENSG00000000460.12  ENST00000286031.6,ENST00000359326.4,ENST000004...   

   22RV1_PROSTATE  2313287_STOMACH  253JBV_URINARY_TRACT  253J_URINARY_TRACT  \
0            5.28             7.01                 22.80               22.88   
1            0.00             0.00                  0.00                0.00   
2           73.38           108.99                 56.51               45.39   
3            9.76            16.76                  2.58                3.25   
4           24.51            13.32                 10.86                5.26   

   42MGBA_CENTRAL_NERVOUS_SYSTEM  5637_URINARY_TR

In [2]:
# transform ENSENBL IDs into official gene symbols

# read gene annotation

# File path
gencode_path = os.path.join(folder, "gencode.v19.annotation.gtf.gz")

if not os.path.exists(gencode_path):
    print("File not found. Please check the file path.")
else:
    print("File exists.")

# Read the gzipped file
gencode_data = pd.read_csv(gencode_path, compression='gzip', sep='\t', comment='#')

# Print the head of the dataset
print(gencode_data.head(10))


File exists.
   chr1   HAVANA        gene  11869  14412  .  + ..1  \
0  chr1   HAVANA  transcript  11869  14409  .  +   .   
1  chr1   HAVANA        exon  11869  12227  .  +   .   
2  chr1   HAVANA        exon  12613  12721  .  +   .   
3  chr1   HAVANA        exon  13221  14409  .  +   .   
4  chr1  ENSEMBL  transcript  11872  14412  .  +   .   
5  chr1  ENSEMBL        exon  11872  12227  .  +   .   
6  chr1  ENSEMBL        exon  12613  12721  .  +   .   
7  chr1  ENSEMBL        exon  13225  14412  .  +   .   
8  chr1  ENSEMBL  transcript  11874  14409  .  +   .   
9  chr1  ENSEMBL        exon  11874  12227  .  +   .   

  gene_id "ENSG00000223972.4"; transcript_id "ENSG00000223972.4"; gene_type "pseudogene"; gene_status "KNOWN"; gene_name "DDX11L1"; transcript_type "pseudogene"; transcript_status "KNOWN"; transcript_name "DDX11L1"; level 2; havana_gene "OTTHUMG00000000961.2";  
0  gene_id "ENSG00000223972.4"; transcript_id "EN...                                                       

In [3]:

# transform ENSENBL IDs into official gene symbols


# Read the GTF file (only necessary columns for gene_id and gene_name)
with gzip.open(gencode_path, 'rt') as f:
    # Parse only rows containing "gene" and extract the relevant column
    gtf_data = pd.read_csv(f, sep='\t', comment='#', header=None, usecols=[8], names=['attributes'])

# Extract gene_id and gene_name from the attributes column
def parse_attributes(attributes):
    """Extract gene_id and gene_name from the attributes string."""
    fields = attributes.split(';')
    gene_id = next((field.split('"')[1] for field in fields if field.strip().startswith("gene_id")), None)
    gene_name = next((field.split('"')[1] for field in fields if field.strip().startswith("gene_name")), None)
    return gene_id, gene_name

# Apply the parsing function to extract gene_id and gene_name
mapping = gtf_data['attributes'].apply(parse_attributes)
gene_mapping_df = pd.DataFrame(mapping.tolist(), columns=['gene_id', 'gene_name'])

# Drop duplicates to ensure unique mapping
gene_mapping_df = gene_mapping_df.drop_duplicates()

# Create a dictionary for mapping
gene_mapping = dict(zip(gene_mapping_df['gene_id'], gene_mapping_df['gene_name']))

# Print the first 10 items of the dictionary
for i, (key, value) in enumerate(gene_mapping.items()):
    print(f"{key}: {value}")
    if i == 9:  # Stop after printing 10 items
        break


ENSG00000223972.4: DDX11L1
ENSG00000227232.4: WASH7P
ENSG00000243485.2: MIR1302-11
ENSG00000237613.2: FAM138A
ENSG00000268020.2: OR4G4P
ENSG00000240361.1: OR4G11P
ENSG00000186092.4: OR4F5
ENSG00000238009.2: RP11-34P13.7
ENSG00000239945.1: RP11-34P13.8
ENSG00000233750.3: CICP27


In [4]:
# Replace gene IDs with gene names using the mapping dictionary
ccle_data['gene_id'] = ccle_data['gene_id'].map(gene_mapping)


# Display the first few rows of the transformed DataFrame
print(ccle_data.head())

    gene_id                                     transcript_ids  \
0    TSPAN6  ENST00000373020.4,ENST00000494424.1,ENST000004...   
1      TNMD                ENST00000373031.4,ENST00000485971.1   
2      DPM1  ENST00000371582.4,ENST00000371583.5,ENST000003...   
3     SCYL3  ENST00000367770.1,ENST00000367771.6,ENST000003...   
4  C1orf112  ENST00000286031.6,ENST00000359326.4,ENST000004...   

   22RV1_PROSTATE  2313287_STOMACH  253JBV_URINARY_TRACT  253J_URINARY_TRACT  \
0            5.28             7.01                 22.80               22.88   
1            0.00             0.00                  0.00                0.00   
2           73.38           108.99                 56.51               45.39   
3            9.76            16.76                  2.58                3.25   
4           24.51            13.32                 10.86                5.26   

   42MGBA_CENTRAL_NERVOUS_SYSTEM  5637_URINARY_TRACT  59M_OVARY  \
0                          23.09               57.94   

In [5]:
# Average duplicates 

# Group by the 'gene_id' column (now containing gene names)
# and calculate the mean for all other columns
averaged_data = ccle_data.groupby('gene_id', as_index=False).mean()

# Display the first few rows of the resulting DataFrame
print(averaged_data.head())


    gene_id  22RV1_PROSTATE  2313287_STOMACH  253JBV_URINARY_TRACT  \
0   5S_rRNA        0.000000         0.000000              0.001429   
1       7SK        0.121429         0.131429              0.075714   
2      A1BG        2.300000         0.140000              0.430000   
3  A1BG-AS1        6.480000         0.120000              0.340000   
4      A1CF        8.590000         3.470000              0.030000   

   253J_URINARY_TRACT  42MGBA_CENTRAL_NERVOUS_SYSTEM  5637_URINARY_TRACT  \
0            0.004286                       0.015714                0.00   
1            0.098571                       0.004286                0.03   
2            1.060000                      49.810000                0.18   
3            1.100000                      16.790000                0.14   
4            0.060000                       0.030000                0.00   

   59M_OVARY  639V_URINARY_TRACT  647V_URINARY_TRACT  ...  \
0   0.011429            0.015714            0.002857  ...   


In [6]:

# read GDSC 1

# File path
gdsc1_path = os.path.join(folder, "GDSC1_fitted_dose_response_27Oct23.xlsx")

if not os.path.exists(gdsc1_path):
    print("File not found. Please check the file path.")
else:
    print("File exists.")

gdsc1_data = pd.read_excel(gdsc1_path)

# Print the head of the dataset
print(gdsc1_data.head(10))

File exists.
  DATASET  NLME_RESULT_ID  NLME_CURVE_ID  COSMIC_ID CELL_LINE_NAME  \
0   GDSC1             342       15580432     684057            ES5   
1   GDSC1             342       15580806     684059            ES7   
2   GDSC1             342       15581198     684062          EW-11   
3   GDSC1             342       15581542     684072        SK-ES-1   
4   GDSC1             342       15581930     687448       COLO-829   
5   GDSC1             342       15585059     687562        8-MG-BA   
6   GDSC1             342       15585789     687568           GB-1   
7   GDSC1             342       15586874     687590        U-87-MG   
8   GDSC1             342       15587948     687600       NCI-H720   
9   GDSC1             342       15590086     687799      NCI-H1648   

  SANGER_MODEL_ID     TCGA_DESC  DRUG_ID  DRUG_NAME PUTATIVE_TARGET  \
0       SIDM00263  UNCLASSIFIED        1  Erlotinib            EGFR   
1       SIDM00269  UNCLASSIFIED        1  Erlotinib            EGFR   
2  

In [7]:
# read GDSC 2

# File path
gdsc2_path = os.path.join(folder, "GDSC2_fitted_dose_response_27Oct23.xlsx")

if not os.path.exists(gdsc2_path):
    print("File not found. Please check the file path.")
else:
    print("File exists.")

gdsc2_data = pd.read_excel(gdsc2_path)

# Print the head of the dataset
print(gdsc2_data.head(10))

File exists.
  DATASET  NLME_RESULT_ID  NLME_CURVE_ID  COSMIC_ID CELL_LINE_NAME  \
0   GDSC2             343       15946310     683667         PFSK-1   
1   GDSC2             343       15946548     684052           A673   
2   GDSC2             343       15946830     684057            ES5   
3   GDSC2             343       15947087     684059            ES7   
4   GDSC2             343       15947369     684062          EW-11   
5   GDSC2             343       15947651     684072        SK-ES-1   
6   GDSC2             343       15947932     687448       COLO-829   
7   GDSC2             343       15948212     687452           5637   
8   GDSC2             343       15948491     687455            RT4   
9   GDSC2             343       15948772     687457          SW780   

  SANGER_MODEL_ID     TCGA_DESC  DRUG_ID     DRUG_NAME PUTATIVE_TARGET  \
0       SIDM01132            MB     1003  Camptothecin            TOP1   
1       SIDM00848  UNCLASSIFIED     1003  Camptothecin            TO

In [8]:
# Ensure both files have the same column names (if needed)
assert list(gdsc1_data.columns) == list(gdsc2_data.columns), "Column names must match!"

# Drop duplicates from the first file if they exist in the second file
# Keeping rows from df2 when duplicates are found
gdsc1_filtered = gdsc1_data[~gdsc1_data[['CELL_LINE_NAME', 'DRUG_ID']].isin(gdsc2_data[['CELL_LINE_NAME', 'DRUG_ID']].to_dict(orient='list')).all(axis=1)]

# Combine the filtered rows from df1 with all rows from df2
combined_gdsc_df = pd.concat([gdsc1_filtered, gdsc2_data], ignore_index=True)

# Display the result
print(combined_gdsc_df.head())


  DATASET  NLME_RESULT_ID  NLME_CURVE_ID  COSMIC_ID CELL_LINE_NAME  \
0   GDSC1             342       15580432     684057            ES5   
1   GDSC1             342       15580806     684059            ES7   
2   GDSC1             342       15581198     684062          EW-11   
3   GDSC1             342       15581542     684072        SK-ES-1   
4   GDSC1             342       15581930     687448       COLO-829   

  SANGER_MODEL_ID     TCGA_DESC  DRUG_ID  DRUG_NAME PUTATIVE_TARGET  \
0       SIDM00263  UNCLASSIFIED        1  Erlotinib            EGFR   
1       SIDM00269  UNCLASSIFIED        1  Erlotinib            EGFR   
2       SIDM00203  UNCLASSIFIED        1  Erlotinib            EGFR   
3       SIDM01111  UNCLASSIFIED        1  Erlotinib            EGFR   
4       SIDM00909          SKCM        1  Erlotinib            EGFR   

     PATHWAY_NAME  COMPANY_ID WEBRELEASE  MIN_CONC  MAX_CONC   LN_IC50  \
0  EGFR signaling        1045          Y  0.007813       2.0  3.966813   
1  E

In [9]:
# Check the shape of the combined DataFrame
num_rows, num_columns = combined_gdsc_df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


Number of rows: 506440
Number of columns: 19


In [11]:
# create data matrix and IC50 vector for each drug

def create_matrix_and_ic50_for_drug(drug_id, combined_gdsc_df, ccle_data):
    """
    Create gene expression data matrix and IC50 vector for a specific DRUG_ID.

    Parameters:
        drug_id (int): The DRUG_ID for which to create the matrix and vector.
        combined_gdsc_df (pd.DataFrame): GDSC dataset with DRUG_ID, CELL_LINE_NAME, and LN_IC50 columns.
        ccle_data (pd.DataFrame): CCLE dataset with gene expression profiles as rows and cell line columns.

    Returns:
        tuple: (gene_expression_matrix, ic50_vector)
            - gene_expression_matrix (pd.DataFrame): Gene expression matrix (G x N, CCLE).
            - ic50_vector (list): Corresponding IC50 values (length N, CCLE).
    """
    # Create mapping from CELL_LINE_ID to CCLE columns
    ccle_columns = {col.split('_')[0]: col for col in ccle_data.columns if '_' in col}

    # Filter rows for the given DRUG_ID
    drug_data = combined_gdsc_df[combined_gdsc_df['DRUG_ID'] == drug_id]

    # Remove duplicate CELL_LINE_NAME entries for this drug
    drug_data = drug_data.groupby('CELL_LINE_NAME', as_index=False).mean()

    # Initialize lists to store matrix columns and IC50 values
    gene_expression_matrix = []
    ic50_vector = []
    matching_cell_lines = []

    # Match CELL_LINE_NAMEs to CCLE columns
    for cell_line in drug_data['CELL_LINE_NAME']:
        if cell_line in ccle_columns:
            ccle_column_name = ccle_columns[cell_line]

            # Append the gene expression profile to the matrix
            gene_expression_matrix.append(ccle_data[ccle_column_name].values)

            # Append IC50 value for this cell line
            ic50_value = drug_data[drug_data['CELL_LINE_NAME'] == cell_line]['LN_IC50'].values[0]
            ic50_vector.append(ic50_value)
            matching_cell_lines.append(cell_line)

    # If no matching cell lines, return None
    if not gene_expression_matrix:
        print(f"No matching cell lines found for DRUG_ID {drug_id}.")
        return None, None

    # Convert the gene expression matrix to a DataFrame with rows as genes and columns as cell lines
    gene_expression_matrix = pd.DataFrame(gene_expression_matrix).T
    gene_expression_matrix.columns = matching_cell_lines

    return gene_expression_matrix, ic50_vector



# Specify a DRUG_ID
drug_id =  1

# Call the function with the specified drug ID
gene_expression_matrix, ic50_vector = create_matrix_and_ic50_for_drug(drug_id, combined_gdsc_df, averaged_data)

# Check the output
if gene_expression_matrix is not None:
    print(f"Gene Expression Matrix for DRUG_ID {drug_id}:")
    print(gene_expression_matrix.head())
    print("\nIC50 Vector:")
    print(ic50_vector[:10])  # Print the first 5 values
else:
    print(f"No data available for DRUG_ID {drug_id}.")



Gene Expression Matrix for DRUG_ID 1:
         697      A101D      A253       CA46         DB        DEL  \
0   0.007143   0.004286  0.001429   0.000000   0.000000   0.000000   
1   0.320000   0.068571  0.147143   2.951429   3.735714   0.214286   
2  41.260000  35.570000  4.620000  41.090000  30.730000  38.100000   
3   8.480000  13.300000  1.990000   7.510000  10.000000   6.000000   
4   0.000000   0.000000  0.000000   0.030000   0.010000   0.020000   

        ECC12       EHEB        EJM       EKVX  ...  RCC10RGB        REH  \
0    0.001429   0.021429   0.284286   0.014286  ...  0.104286   0.000000   
1    0.061429   2.254286   0.622857   0.064286  ...  0.037143   1.175714   
2  111.200000  27.200000  11.610000  16.270000  ...  0.190000  37.410000   
3   20.410000  16.100000   6.640000   3.210000  ...  0.300000  10.420000   
4   46.130000   0.040000   0.000000   0.020000  ...  0.040000   0.000000   

        RKO        RL     SF126      SF268      SF539       SIMA  SNB75  \
0  0.0000